In [2]:
# Just to define file paths, not related to sisl
from pathlib import Path

# Sisl imports
import sisl
import sisl.viz
import numpy as np
import scipy

# To quickly plot the hamiltonian matrix
import plotly.express as px

import numpy as np
import torch
import yaml
import importlib
import random
import matplotlib.pyplot as plt

# So that we can plot sisl geometries
import sisl.viz

from e3nn import o3
from pathlib import Path

from graph2mat import (
    PointBasis,
    BasisTableWithEdges,
    BasisConfiguration,
    MatrixDataProcessor,
)
from graph2mat.bindings.torch import TorchBasisMatrixData, TorchBasisMatrixDataset
from graph2mat.bindings.e3nn import E3nnGraph2Mat

def load_config(path="../config.yaml"):
    with open(path, "r") as f:
        return yaml.safe_load(f)
    
def flatten(xss):
    return [x for xs in xss for x in xs]

#Get the fdf file for the calculation
path_uc = Path("../dataset/SHARE_OUTPUTS_8_ATOMS/0a2a-fbca-4649-8012-f5aa640bfd1d")   #SIESTA calculation folder
fdf_uc = sisl.get_sile(path_uc / "aiida.HSX")
geometry = fdf_uc.read_geometry()
H_uc = fdf_uc.read_hamiltonian()
cell=geometry.cell

def reduced_coord(kpt, cell):
    """Convert a k-point to reduced coordinates."""
    return (cell.T)@kpt/(2*np.pi)

/home/alapena/miniconda3/envs/g2m_upd/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indices = torch.

In [55]:
# We will rely for now on the shifts generated by siesta. Because their computation would require a lot of effort. However, we should be able to compute them by just knowing the atom orbital positions and types, which we presumably know.

# # TODO: Adapt this to sparse format for matrix_uc
# def reconstruct_tim(matrix_uc, k_point, cell, isc_list, isc_idx_list):
#     n_rows = matrix_uc.shape[0] 
#     n_cols = matrix_uc.shape[1]  
#     n_shifts = n_cols // n_rows  

#     # Get the indices of the shifts

#     tim_manual = np.zeros((n_rows, n_rows), dtype=np.complex128)
#     blocks = []
#     for isc_idx in isc_idx_list:
#         # Compute the actual shift, not only the "integer jumps"
#         isc = isc_list[isc_idx]
#         shift = np.array(np.sum([cell[i] * isc[i] for i in range(len(isc))], axis=0))

#         block = matrix_uc[0:n_rows, isc_idx:isc_idx+n_rows] * np.exp(-1j*np.dot(k_point, shift))
#         tim_manual +=  block # sum all n_rows x n_rows blocks of H_matrix (at k=0, all phase factors are 1)
#         block[block == 0] = None  # replace zeros with None for better visualization
#         blocks.append(block)
    
#     return tim_manual

def ReconstructH(k_point, H_coo, orb_i, orb_j, isc, cell):

    no=H_coo.shape[0]    #number of rows of H_matrix, i.e. number of orbitals
    H_g=np.zeros((no, no), dtype=complex) #no x no matrix of zeros for our H(k) which we wil now fill (it may be wasteful to use a dense matrix for that, but we do it here for simplicity)

    for k in range(0, H_coo.nnz):
        R_uc = np.array(np.sum([cell[i] * isc[k, i] for i in range(len(isc[k]))], axis=0))  #unit cell to unit cell repetition distance (i.e. cell phase convention)
        Phase=np.exp(1j*k_point.dot(R_uc))
        H_g[orb_i[k],orb_j[k]]+=H_coo.data[k]*Phase   #add contribution to correct entry of H(k)

    return H_g

# def reconstruct_tim_from_coo(k_point, h_coo, isc_list, cell):
#     no=h_coo.shape[0]
#     hk=np.zeros((no, no), dtype=complex)
#     for k in range(h_coo.nnz):
#         col = h_coo.col[k] # The column where k is located in h_coo
#         orb_j = col - 13*(col//13) # Its corresponding orbital in the uc 
#         orb_i = h_coo.row[k]
#         isc = isc_list[col] # Its corresponding isc 

#         # Phase factor:
#         r_uc = np.array(np.sum([cell[i] * isc[i] for i in range(len(isc))], axis=0))
#         phase=np.exp(1j*k_point.dot(r_uc))

#         hk[orb_i, orb_j] += h_coo.data[k]*phase

#     return hk

def reconstruct_tim_from_coo(k_point, M_coo, geometry, cell):
    no = M_coo.shape[0]  # Number of orbitals in the unit cell
    H_g = np.zeros((no, no), dtype=complex)  # Output matrix
    
    for k in range(M_coo.nnz):
        # Extract row index (already in unit cell basis)
        i_uc = M_coo.row[k]
        # Map supercell column index to unit cell orbital and shift
        j_uc = geometry.osc2uc(M_coo.col[k])
        isc_k = geometry.o2isc(M_coo.col[k])
        
        # Compute lattice vector for this shift
        R_uc = np.sum([cell[d] * isc_k[d] for d in range(3)], axis=0)
        Phase = np.exp(1j * k_point.dot(R_uc))
        
        # Accumulate value with phase
        H_g[i_uc, j_uc] += M_coo.data[k] * Phase
    
    return H_g


H_coo = H_uc.tocsr().tocoo()  # Convert the Hamiltonian to COO format
S_uc = fdf_uc.read_overlap() 
S_coo = S_uc.tocsr().tocoo()  # Convert the Overlap to COO format

# Orbital indices and shifts extraction
orb_i=-np.ones(H_coo.nnz, dtype=int) #we fill everything with minus ones to be able to detect iif something went wrong
orb_j=-np.ones(H_coo.nnz, dtype=int) #same
isc=-np.ones((H_coo.nnz,3), dtype=int) #same

orb_i=H_coo.row
for k in range(H_coo.nnz):
    orb_j[k]=geometry.osc2uc(H_coo.col[k])   #unit cell orbital indices corresponding to the "supercell" indices
    isc[k]=geometry.o2isc(H_coo.col[k])      #which unit cell / shift that orbital belongs to wrt. the (0,0,0) unit cell

# K path
kzs=np.linspace(0,1, num=50)
kdir=geometry.rcell[:,2]
k_path=np.array([kz*kdir for kz in kzs])

# TIM reconstruction
energy_bands = []
for k_point in k_path:
    Hk = ReconstructH(k_point, H_coo, orb_i, orb_j, isc, cell)
    Sk = ReconstructH(k_point, S_coo, orb_i, orb_j, isc, cell)

    Ek = scipy.linalg.eigh(Hk, Sk, eigvals_only=True)

    energy_bands.append(Ek)

    

KeyboardInterrupt: 

In [26]:
# Get unique ordered isc_list:
n_cols = H_coo.shape[1]
isc_list = [geometry.o2isc(io) for io in range(n_cols)]
unique_isc_list, isc_idx_list = np.unique(isc_list, return_index=True, axis=0)
isc_idx_list = sorted(isc_idx_list)
unique_isc_list = [isc_list[index] for index in isc_idx_list]

In [ ]:
print(H_coo.shape)
H_coo.col.shape
reconstruct_tim_from_coo(k_point, H_coo, geometry, cell)

(104, 14040)


NameError: name 'H_g' is not defined

In [21]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_columns(array_pred, array_true=None, x=None, titles=None, xlabel=None, ylabel=None, title=None):
    """
    Plot each column of a 2D numpy array as separate traces.
    
    Parameters:
    - array: 2D numpy array to plot (each column will be a separate trace)
    - x: Optional x-axis values (if None, uses array indices)
    - titles: Optional list of names for each column/trace
    - xlabel: Label for x-axis
    - ylabel: Label for y-axis
    - title: Title for the overall plot
    """
    if array_pred.ndim != 2:
        raise ValueError("Input array must be 2-dimensional")
        
    num_cols = array_pred.shape[1]
    
    if titles is None:
        titles = [f'Column {i+1}' for i in range(num_cols)]
    elif len(titles) != num_cols:
        raise ValueError("Number of titles must match number of columns")
    
    if x is None:
        x = np.arange(array_pred.shape[0])
    elif len(x) != array_pred.shape[0]:
        raise ValueError("Length of x must match number of rows in array")
    
    fig = go.Figure()
    
    for col in range(num_cols):
        fig.add_trace(go.Scatter(
            x=x,
            y=array_pred[:, col],
            mode='markers',
            name=titles[col],
            line=dict(color='red')
        ))

        if array_true is not None:
            fig.add_trace(go.Scatter(
                x=x,
                y=array_true[:, col],
                mode='lines',
                name="True " + titles[col],
                line=dict(color='red')
            ))
    
    fig.update_layout(
        title=title,
        xaxis_title=xlabel,
        yaxis_title=ylabel,
        hovermode='x unified',
        height=800,
        xaxis_title_standoff=15,
    )

    fig.update_xaxes(
        showticklabels=False,
        # tickvals=x[::2],
    )
    
    return fig

# Plot the energy bands
plot_columns(energy_bands_plot[:,0:10], array_true=energy_bands_plot[:,0:10], x=[k_path[:,2]]*50, xlabel="k", ylabel="Enegry (eV)", title="Energy bands")

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'line': {'color': 'red'},
              'mode': 'markers',
              'name': 'Column 1',
              'type': 'scatter',
              'x': [[0.0, 0.01846017779121184, 0.03692035558242368,
                    0.05538053337363552, 0.07384071116484736, 0.0923008889560592,
                    0.11076106674727104, 0.12922124453848288, 0.14768142232969472,
                    0.16614160012090656, 0.1846017779121184, 0.20306195570333024,
                    0.22152213349454208, 0.2399823112857539, 0.25844248907696576,
                    0.2769026668681776, 0.29536284465938945, 0.3138230224506013,
                    0.3322832002418131, 0.35074337803302497, 0.3692035558242368,
                    0.38766373361544865, 0.4061239114066605, 0.4245840891978723,
                    0.44304426698908417, 0.46150444478029606, 0.4799646225715078,
                    0.4984248003627197, 0.5168849781539315, 0.5353451559451434,
                    0.5538053337363552, 0.572265511527567, 0.5907256893187789,
                    0.6091858671099908, 0.6276460449012026, 0.6461062226924144,
                    0.6645664004836263, 0.683026578274838, 0.7014867560660499,
                    0.7199469338572617, 0.7384071116484736, 0.7568672894396854,
                    0.7753274672308973, 0.7937876450221092, 0.812247822813321,
                    0.8307080006045328, 0.8491681783957447, 0.8676283561869566,
                    0.8860885339781683, 0.9045487117693802], [0.0,
                    0.01846017779121184, 0.03692035558242368, 0.05538053337363552,
                    0.07384071116484736, 0.0923008889560592, 0.11076106674727104,
                    0.12922124453848288, 0.14768142232969472, 0.16614160012090656,
                    0.1846017779121184, 0.20306195570333024, 0.22152213349454208,
                    0.2399823112857539, 0.25844248907696576, 0.2769026668681776,
                    0.29536284465938945, 0.3138230224506013, 0.3322832002418131,
                    0.35074337803302497, 0.3692035558242368, 0.38766373361544865,
                    0.4061239114066605, 0.4245840891978723, 0.44304426698908417,
                    0.46150444478029606, 0.4799646225715078, 0.4984248003627197,
                    0.5168849781539315, 0.5353451559451434, 0.5538053337363552,
                    0.572265511527567, 0.5907256893187789, 0.6091858671099908,
                    0.6276460449012026, 0.6461062226924144, 0.6645664004836263,
                    0.683026578274838, 0.7014867560660499, 0.7199469338572617,
                    0.7384071116484736, 0.7568672894396854, 0.7753274672308973,
                    0.7937876450221092, 0.812247822813321, 0.8307080006045328,
                    0.8491681783957447, 0.8676283561869566, 0.8860885339781683,
                    0.9045487117693802], [0.0, 0.01846017779121184,
                    0.03692035558242368, 0.05538053337363552, 0.07384071116484736,
                    0.0923008889560592, 0.11076106674727104, 0.12922124453848288,
                    0.14768142232969472, 0.16614160012090656, 0.1846017779121184,
                    0.20306195570333024, 0.22152213349454208, 0.2399823112857539,
                    0.25844248907696576, 0.2769026668681776, 0.29536284465938945,
                    0.3138230224506013, 0.3322832002418131, 0.35074337803302497,
                    0.3692035558242368, 0.38766373361544865, 0.4061239114066605,
                    0.4245840891978723, 0.44304426698908417, 0.46150444478029606,
                    0.4799646225715078, 0.4984248003627197, 0.5168849781539315,
                    0.5353451559451434, 0.5538053337363552, 0.572265511527567,
                    0.5907256893187789, 0.6091858671099908, 0.6276460449012026,
                    0.6461062226924144, 0.6645664004836263, 0.683026578274838,
                    0.7014867560660499, 0.7199469338572617, 0.7384071116484736,
                    0.7568672894396854, 0.7753274672308973, 0.7937876450221092,
           

In [ ]:
# === List of paths to all structures ===
parent_path = Path('../dataset')
n_atoms_paths = list(parent_path.glob('*/'))
paths = []
for n_atoms_path in n_atoms_paths:
    structure_paths = list(n_atoms_path.glob('*/'))
    paths.append(structure_paths)
paths = flatten(paths)

random.seed(42)
random.shuffle(paths)

# Test TIM reconstruction in sparse format

In [57]:
k_point = np.array([3,0,5])
# atol=None
# rtol=None

tim_sisl = H_uc.Hk(reduced_coord(k_point, cell), gauge='cell').toarray()
# tim_manual = reconstruct_tim(H_uc.tocsr().todense(), k_point, cell, isc_list, isc_idx_list)
tim_manual = reconstruct_tim_from_coo(k_point, H_coo, geometry, cell)

tim_sisl_plot = np.abs(tim_sisl)
tim_manual_plot = np.abs(tim_manual)

are_close = np.allclose(tim_sisl, tim_manual)

print(are_close)
px.imshow(tim_sisl_plot, title="SISL TIM at Gamma").show()
px.imshow(tim_manual_plot, title="Manual TIM at Gamma").show()

True


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

# Silly tests

In [ ]:
from scipy.sparse import coo_matrix

matrix = np.array([[1,0,2], [0,1,0], [3,0,5]])
matrix = coo_matrix(matrix)
print(matrix.col)

[0 2 1 0 2]
